In [ ]:
import torch
import torch.nn as nn

# Original ConvLSTM cell as proposed by Shi et al.
class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTMCell, self).__init__()  

        if activation == "tanh":
            self.activation = torch.tanh 
        elif activation == "relu":
            self.activation = torch.relu
        
        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        self.conv = nn.Conv2d(
            in_channels=in_channels + out_channels, 
            out_channels=4 * out_channels, 
            kernel_size=kernel_size, 
            padding=padding)           

        # Initialize weights for Hadamard Products
        self.W_ci = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_co = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_cf = nn.Parameter(torch.Tensor(out_channels, *frame_size))

    def forward(self, X, H_prev, C_prev):

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)

        input_gate = torch.sigmoid(i_conv + self.W_ci * C_prev )
        forget_gate = torch.sigmoid(f_conv + self.W_cf * C_prev )

        # Current Cell output
        C = forget_gate*C_prev + input_gate * self.activation(C_conv)

        output_gate = torch.sigmoid(o_conv + self.W_co * C )

        # Current Hidden State
        H = output_gate * self.activation(C)

        return H, C

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ConvLSTM(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTM, self).__init__()

        self.out_channels = out_channels

        # We will unroll this over time steps
        self.convLSTMcell = ConvLSTMCell(in_channels, out_channels, 
        kernel_size, padding, activation, frame_size)

    def forward(self, X):

        # X is a frame sequence (batch_size, num_channels, seq_len, height, width)

        # Get the dimensions
        batch_size, _, seq_len, height, width = X.size()

        # Initialize output
        output = torch.zeros(batch_size, self.out_channels, seq_len, 
        height, width, device=device)
        
        # Initialize Hidden State
        H = torch.zeros(batch_size, self.out_channels, 
        height, width, device=device)

        # Initialize Cell Input
        C = torch.zeros(batch_size,self.out_channels, 
        height, width, device=device)

        # Unroll over time steps
        for time_step in range(seq_len):

            H, C = self.convLSTMcell(X[:,:,time_step], H, C)

            output[:,:,time_step] = H

        return output

In [ ]:
class Seq2Seq(nn.Module):

    def __init__(self, num_channels, num_kernels, kernel_size, padding, 
    activation, frame_size, num_layers):

        super(Seq2Seq, self).__init__()

        self.sequential = nn.Sequential()

        # Add First layer (Different in_channels than the rest)
        self.sequential.add_module(
            "convlstm1", ConvLSTM(
                in_channels=num_channels, out_channels=num_kernels,
                kernel_size=kernel_size, padding=padding, 
                activation=activation, frame_size=frame_size)
        )

        self.sequential.add_module(
            "batchnorm1", nn.BatchNorm3d(num_features=num_kernels)
        ) 

        # Add rest of the layers
        for l in range(2, num_layers+1):

            self.sequential.add_module(
                f"convlstm{l}", ConvLSTM(
                    in_channels=num_kernels, out_channels=num_kernels,
                    kernel_size=kernel_size, padding=padding, 
                    activation=activation, frame_size=frame_size)
                )
                
            self.sequential.add_module(
                f"batchnorm{l}", nn.BatchNorm3d(num_features=num_kernels)
                ) 

        # Add Convolutional Layer to predict output frame
        self.conv = nn.Conv2d(
            in_channels=num_kernels, out_channels=num_channels,
            kernel_size=kernel_size, padding=padding)

    def forward(self, X):

        # Forward propagation through all the layers
        output = self.sequential(X)

        # Return only the last output frame
        output = self.conv(output[:,:,-1])
        
        return nn.Sigmoid()(output)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam

from torch.utils.data import DataLoader


# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = Seq2Seq(num_channels=1, num_kernels=64, 
kernel_size=(3, 3), padding=(1, 1), activation="relu", 
frame_size=(64,64), num_layers=1).to(device)
#model_state_dict = torch.load('drive/MyDrive/model.pth')
#model.load_state_dict(model_state_dict)

optim = Adam(model.parameters(), lr=1e-4)

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.MSELoss (reduction="sum")

In [ ]:
# Load Data as Numpy Array
from torch.utils.data import DataLoader
MovingMNIST = np.load('drive/MyDrive/data_new.npy').astype(np.float32)

train_data = MovingMNIST[0:18000] 
val_data = MovingMNIST[18000:19000]
test_data = MovingMNIST[19000:20000]  


In [ ]:
def collate(batch):

    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)                    
    batch = batch.to(device)                     
    # Randomly pick 10 frames as input, 11th frame is target
    rand = np.random.randint(10,20)                     
    return batch[:,:,rand-10:rand], batch[:,:,rand]     


train_loader = DataLoader(train_data,shuffle=True, 
                         batch_size=16, collate_fn=collate)

# Test Data Loader
val_loader = DataLoader(val_data,shuffle=True, 
                         batch_size=16, collate_fn=collate)



In [ ]:
import sys
train_loss_l=[]
val_loss_l=[]
runnning_mae_train_l=[]
runnning_mse_train_l=[]
runnning_mae_val_l=[]
runnning_mse_val_l=[]

In [ ]:
num_epochs = 20
import math
for epoch in range(1, num_epochs+1):
    runnning_mae_train=0
    runnning_mse_train=0
    train_loss = 0                                                 
    model.train()                                                  
    for batch_num, (input, target) in enumerate(train_loader, 1):  
        output = model(input)                                  
        loss = criterion(output.flatten(), target.flatten())       
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()                                           
        train_loss += loss.item() 
        error = torch.abs(output.flatten() - target.flatten()).sum().data
        squared_error = ((output.flatten() - target.flatten())*(output.flatten() - target.flatten())).sum().data
        runnning_mae_train += error
        runnning_mse_train += squared_error
      
    train_loss /= len(train_loader.dataset)
    runnning_mae_train /= len(train_loader.dataset) 
    runnning_mse_train = math.sqrt(runnning_mse_train / len(train_loader.dataset))                     
    val_loss = 0  
    runnning_mae_val=0
    runnning_mse_val=0                                               
    model.eval()                                                   
    with torch.no_grad():                                          
        for input, target in val_loader:                          
            output = model(input)                                
            loss = criterion(output.flatten(), target.flatten())   
            val_loss += loss.item()
            error = torch.abs(output.flatten() - target.flatten()).sum().data
            squared_error = ((output.flatten() - target.flatten())*(output.flatten() - target.flatten())).sum().data
            runnning_mae_val += error
            runnning_mse_val += squared_error                              
    val_loss /= len(val_loader.dataset)  
    runnning_mae_val /= len(val_loader.dataset) 
    runnning_mse_val = math.sqrt(runnning_mse_val / len(val_loader.dataset) )                          
    torch.save(model.state_dict(),f'drive/MyDrive/model_{epoch}.pth')
    print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f} MAE_train: {:.8} MSE_train:{:.8} MAE_val: {:.8} MSE_val:{:.8}\n".format(
        epoch, train_loss, val_loss,runnning_mae_train,runnning_mse_train,runnning_mae_val,runnning_mse_val))
    train_loss_l.append(train_loss)
    val_loss_l.append(val_loss)
    runnning_mae_train_l.append(runnning_mae_train)
    runnning_mse_train_l.append(runnning_mse_train)
    runnning_mae_val_l.append(runnning_mae_val)
    runnning_mse_val_l.append(runnning_mse_val)
    np.save("drive/MyDrive/train_loss_l.npy",np.array(train_loss_l))
    np.save("drive/MyDrive/val_loss_l.npy",np.array(val_loss_l))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  after removing the cwd from sys.path.


Epoch:1 Training Loss:206.71 Validation Loss:179.55 MAE_train: 470.3316 MSE_train:14.377335 MAE_val: 359.65225 MSE_val:13.39954

Epoch:2 Training Loss:178.14 Validation Loss:174.04 MAE_train: 374.41104 MSE_train:13.346883 MAE_val: 371.00891 MSE_val:13.19234

Epoch:3 Training Loss:173.53 Validation Loss:167.83 MAE_train: 363.41245 MSE_train:13.173269 MAE_val: 356.43884 MSE_val:12.954769

Epoch:4 Training Loss:171.41 Validation Loss:169.76 MAE_train: 357.70255 MSE_train:13.092456 MAE_val: 340.77963 MSE_val:13.029192

Epoch:5 Training Loss:168.36 Validation Loss:170.07 MAE_train: 349.43109 MSE_train:12.975414 MAE_val: 346.92786 MSE_val:13.040998

Epoch:6 Training Loss:165.21 Validation Loss:166.78 MAE_train: 341.85593 MSE_train:12.853466 MAE_val: 330.64008 MSE_val:12.914156

Epoch:7 Training Loss:165.82 Validation Loss:166.63 MAE_train: 342.05682 MSE_train:12.877195 MAE_val: 349.74814 MSE_val:12.90862

Epoch:8 Training Loss:166.40 Validation Loss:163.61 MAE_train: 342.27063 MSE_train:12.8

In [ ]:

def collate_test(batch):

    # Last 10 frames are target
    target = np.array(batch)[:,10:]                     
    
    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)                                    
    batch = batch.to(device)                          
    return batch, target

# Test Data Loader
test_loader = DataLoader(test_data,shuffle=True, 
                         batch_size=1000, collate_fn=collate_test)

# Get a batch
batch, target = next(iter(test_loader))

# Initialize output sequence
output = np.zeros(target.shape, dtype=np.uint8)

# Loop over timesteps
for timestep in range(target.shape[1]):
  input = batch[:,:,timestep:timestep+10]   
  output[:,timestep]=(model(input).squeeze(1).cpu()>0.5)*255.0

NameError: ignored

In [ ]:
for tgt, out in zip(target, output):       # Loop over samples
    
    # Write target video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, tgt, "GIF", fps = 1)    
        target_gif = gif.getvalue()

    # Write output video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, out, "GIF", fps = 1)    
        output_gif = gif.getvalue()

    display(HBox([widgets.Image(value=target_gif), 
                  widgets.Image(value=output_gif)]))